# Designing a sales dashboard

![Cello](data/images/Cello_study.jpg)

## Goals

In this case, you will learn how to plan a visual dashboard. You will expand upon the skills you learned about choosing visualizations and presenting them in a clear and compelling way. The visualization techniques and concepts we will introduce today are:

* Dashboards
* Gauges
* Bubble charts
* Choropleth maps
* Scorecards
* Dimensions
* Metrics
* Dashboard pages
* Drill down

## Introduction

**Business Context.** You are a member of the business analytics team of *Simply Music*, a Florida-based musical instrument store that has been growing at a fast pace over the last few years. Management is keen to find out what the most popular items are and which locations are the most profitable in order to adjust their marketing strategies and their stock logistics to increase sales and revenue. They also want to learn about the number of deliveries made outside Florida since they are contemplating opening a new branch in another state.

**Business Problem.** You are tasked with creating a sales dashboard for management to use. This dashboard must display sales, revenue, and profit over time and allow for queries across product, location, and customer age. From your previous meetings with the management, you know that they are interested in having these three broad questions answered:

1. How have our sales, revenue, and profits evolved over time?
2. What are the locations with the highest numbers of sales and profit?
3. What are the items with the highest numbers of sales, profit, and revenue across product description, product type, product line, location, customer age group, and state of delivery?

**Analytical Context.** You have been given a sample dataset of 5,000 sales records from Simply Music corresponding to the years 2018 and 2019. They ask you to create the dashboard with this sample data, and after this is done, and your dashboard receives approval, your team will have to connect it to an online database to feed it the real-time data that comes from the company's sales tracking software. The sample table and the database share the same structure, so you don't have to worry about database management or creation.

## Understanding the business process

Data professionals are often required to extract valuable information from datasets without being given specific questions to answer. We usually address these cases with exploratory data analysis combined with domain knowledge. Other times, we can get the clients to articulate a limited set of questions. There is a third scenario, where the clients want to explore the data themselves in such a way that they can easily answer *ad-hoc* queries that can't possibly be known in advance. This is much more open-ended and requires us to intimately understand the natural structure of the data and likely queries.

One way to address this need is to create **dashboards** that organize and display all relevant business information along with various filters that users might want to have, while being fully extensible if required in the future. When you create dashboards, you define a set of relevant variables and feed them into interactive visualizations that the users can then filter at will to explore relationships among the variables. Here are some examples:

![Dashboard 1](data/images/d1.png)
<br><br><br>
![Dashboard 2](data/images/d2.png)
<br><br><br>
![Dashboard 3](data/images/d3.png)

Dashboards work best when they mirror the business processes that underlie the data. It is *always* advised that you study the greater business context so that you can put yourself into the shoes of your end users (in this case, management) and deliver a product that is user-friendly and pertinent.

### Specifics of *Simply Music*

![Logo](data/images/logo.jpg)

Simply Music is a retailer that sells mostly to individual buyers who buy in small quantities. The customers are not restricted to a single state, and in fact, many of them buy online from all over the country. But they also have three physical shops located in the state of Florida. These are the product lines that the store serves:

1. Musical instruments
    1. Acoustic pianos
    2. Digital pianos and keyboards
    3. Guitars (electric and acoustic)
    4. Woodwinds (flutes, oboes, bassoons, saxophones, etc.)
    5. Brass (trumpets, horns)
    6. Synthesizers
    7. Strings (everything bowed: violins, cellos, etc.)
    8. Percussion (mostly drum kits)
2. Accessories
    1. Strings (for guitars, violins, etc.)
    2. Pedals and amps (for electric guitars)
    3. Sheet music

Furthermore, these are the four selling channels that the company uses:

1. Physical store in Fort Lauderdale
2. Physical store in Tampa
3. Physical store in West Palm Beach
4. Online store

Each transaction is recorded at the time of purchasing, whether the purchase happens online or at a physical location. Accepted means of payment include cash and credit card. If the customer is not happy with their product, they can file a complaint within 10 days of the purchase.

## Looking at the data

The database team has given us access to a sample of 5,000 transactions and some other relevant data.

The first table is called [`transactions`](data/transactions.csv) (here and in the following tables, we only show the first rows):

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>TIME_KEY</th>      <th>PRODUCT_KEY</th>      <th>UNITS_SOLD</th>      <th>DOLLARS_SOLD</th>      <th>CHANNEL</th>      <th>CUSTOMER_KEY</th>      <th>DESTINATION_STATE</th>    </tr>  </thead>  <tbody>    <tr>      <td>20191104</td>      <td>339695</td>      <td>1</td>      <td>5800</td>      <td>Fort Lauderdale</td>      <td>1830</td>      <td>NaN</td>    </tr>    <tr>      <td>20190512</td>      <td>293442</td>      <td>1</td>      <td>14</td>      <td>Website</td>      <td>4814</td>      <td>US-Colorado</td>    </tr>    <tr>      <td>20180905</td>      <td>239457</td>      <td>1</td>      <td>3509</td>      <td>Website</td>      <td>5185</td>      <td>US-Massachussets</td>    </tr>    <tr>      <td>20191121</td>      <td>392001</td>      <td>1</td>      <td>1899</td>      <td>Fort Lauderdale</td>      <td>3388</td>      <td>NaN</td>    </tr>    <tr>      <td>20180613</td>      <td>443553</td>      <td>1</td>      <td>159</td>      <td>Website</td>      <td>7557</td>      <td>US-Delaware</td>    </tr>  </tbody></table>

There are several attributes in this table, namely:

1. **TIME_KEY**: A key that refers to the year, month, and day of the purchase in YYMMDD format (e.g., `180122` means January 22, 2018)
2. **PRODUCT_KEY**: A number that uniquely identifies the product
3. **UNITS_SOLD**: The number of units sold
4. **DOLLARS_SOLD**: The total amount of dollars sold for this product. This variable is equal to the units sold times the unit price
5. **CHANNEL**: Either `Fort Lauderdale`, `Tampa`, `West Palm Beach` or `Website`
6. **CUSTOMER_KEY**: A number that uniquely identifies the customer
7. **DESTINATION_STATE**: The state to which the customer wants their products delivered. Available only for online transactions

Additionally, we also have the [`products`](data/products.csv) table:

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>PRODUCT_KEY</th>      <th>PRODUCT_LINE</th>      <th>PRODUCT_TYPE</th>      <th>DESCRIPTION</th>      <th>COST</th>      <th>UNIT_PRICE</th>    </tr>  </thead>  <tbody>    <tr>      <td>441354</td>      <td>Musical instruments</td>      <td>Digital pianos</td>      <td>Casio Celviano AP270</td>      <td>776.26</td>      <td>1049</td>    </tr>    <tr>      <td>460568</td>      <td>Musical instruments</td>      <td>Digital pianos</td>      <td>Casio Celviano AP470</td>      <td>1139.24</td>      <td>1499</td>    </tr>    <tr>      <td>451845</td>      <td>Musical instruments</td>      <td>Digital pianos</td>      <td>Casio Celviano AP650</td>      <td>1696.50</td>      <td>2175</td>    </tr>    <tr>      <td>270105</td>      <td>Musical instruments</td>      <td>Digital pianos</td>      <td>Casio Celviano AP700</td>      <td>1874.25</td>      <td>2499</td>    </tr>    <tr>      <td>339695</td>      <td>Musical instruments</td>      <td>Digital pianos</td>      <td>Yamaha CLP 785</td>      <td>4118.00</td>      <td>5800</td>    </tr>  </tbody></table>

The attributes are:

1. **PRODUCT_KEY**: The unique identifier of the product
2. **PRODUCT_LINE**: The product line (either "Musical instruments" or "Accessories")
3. **PRODUCT_TYPE**: The type of the product (e.g., "Acoustic pianos", "Brass")
4. **DESCRIPTION**: The name of the product (e.g., "Casio Digital Piano AP650")
5. **COST**: How much the company estimates the item costs them (includes fixed and variable costs) - this is also known as the unit cost
6. **UNIT_PRICE**: The unit price

Let's also inspect the [`time`](data/time.csv) and [`customers`](data/customers.csv) tables:

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>TIME_KEY</th>      <th>YEAR</th>      <th>MONTH</th>      <th>DAY</th>      <th>WEEK_OF_YEAR</th>      <th>QUARTER</th>    </tr>  </thead>  <tbody>    <tr>      <td>20180101</td>      <td>2018</td>      <td>1</td>      <td>1</td>      <td>1</td>      <td>1</td>    </tr>    <tr>      <td>20180102</td>      <td>2018</td>      <td>1</td>      <td>2</td>      <td>1</td>      <td>1</td>    </tr>    <tr>      <td>20180103</td>      <td>2018</td>      <td>1</td>      <td>3</td>      <td>1</td>      <td>1</td>    </tr>    <tr>      <td>20180104</td>      <td>2018</td>      <td>1</td>      <td>4</td>      <td>1</td>      <td>1</td>    </tr>    <tr>      <td>20180105</td>      <td>2018</td>      <td>1</td>      <td>5</td>      <td>1</td>      <td>1</td>    </tr>  </tbody></table>

The attributes of the `time` table are:

1. **TIME_KEY**: The time key in YYYYMMDD format (e.g., `20180122` means January 22, 2018)
2. **YEAR**: The year
3. **MONTH**: The month in numerical format (e.g., `2` means February)
4. **DAY**: The day in DD format
5. **WEEK_OF_YEAR**: The week of the year (the week of January 1st counts as week 1)
6. **QUARTER**: The quarter to which the month belongs

Notice that we don't have data for hours or minutes.

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>CUSTOMER_KEY</th>      <th>CUSTOMER_NAME</th>      <th>CUSTOMER_AGE</th>      <th>CUSTOMER_AGE_GROUP</th>    </tr>  </thead>  <tbody>    <tr>      <td>3669</td>      <td>Terrance Griffin</td>      <td>85</td>      <td>80, 90</td>    </tr>    <tr>      <td>5282</td>      <td>Alec Cordova</td>      <td>58</td>      <td>50, 60</td>    </tr>    <tr>      <td>2267</td>      <td>Mariana Mccarty</td>      <td>70</td>      <td>60, 70</td>    </tr>    <tr>      <td>6113</td>      <td>Annalise Richmond</td>      <td>95</td>      <td>90, 100</td>    </tr>    <tr>      <td>6938</td>      <td>Ernest Gay</td>      <td>87</td>      <td>80, 90</td>    </tr>  </tbody></table>

The fields of the `customers` table are:

1. **CUSTOMER_KEY**: The unique identifier of the customer (can be a social security number or any other number)
2. **CUSTOMER_NAME**: The customer's full name
3. **CUSTOMER_AGE**: The age of the customer
4. **CUSTOMER_AGE_GROUP**: The customer's age group

You might be wondering how we're going to pool all this data in order to create our dashboard. The good news is that dashboarding tools usually offer easy ways to merge tables, so in many scenarios, you don't have to worry too much about that. The other good news is that in this particular case, we've already pre-merged the data for you in the [`simply_music.csv`](data/simply_music.csv) dataset.

## Choosing the right visualizations

Let's keep in mind our problem statement: "The dashboard must display sales, revenue, and profit across product type, location, and customer age." What visualizations are most appropriate given this?

You might recall that data can be *categorical* or *numerical*. In turn, numerical data can be of type *integer* or *float*. Data types are probably the key criterion to consider when it comes to choosing a visualization.

### Exercise 1

Which of the following three chart types are appropriate for plotting categorical vs. numerical data? And which works best for numerical vs. numerical data?

**Gauges**:
<img src="data/images/gauge.jpg" width="300"/>

**Choropleth maps**:

<img src="data/images/choropleth.jpg" width="400"/>

**Scatter plots**:

<img src="data/images/scatter_wiki.jpg" width="400"/>

<ul>
A. Numerical vs. numerical: Gauges, choropleth maps; Categorical vs. numerical: Scatter plots<br>
B. Categorical vs. numerical: Gauges, choropleth maps; Numerical vs. numerical: Scatter plots<br>
C. Categorical vs. numerical: Scatter plots, choropleth maps; Numerical vs. numerical: Gauges<br>
D. Numerical vs. numerical: Scatter plots, choropleth maps; Categorical vs. numerical: Gauges<br>
</ul>

**Answer.**

Expanding on the above ideas, you can even add a third numerical variable and a categorical variable into the mix and create a **bubble chart**, in which the third numerical variable is represented as the size of the bubble and the categorical variable maps to a color code. An interactive example is this bubble chart from [Google Public Data Explorer](https://www.google.com/publicdata/explore?ds=d5bncppjof8f9_&ctype=b&strail=false&nselm=s&met_x=sp_dyn_le00_in&scale_x=lin&ind_x=false&met_y=sp_dyn_tfrt_in&scale_y=lin&ind_y=false&met_s=sp_pop_totl&scale_s=lin&ind_s=false&dimp_c=country:region&ifdim=country&hl=en_US&dl=en_US&iconSize=0.5&uniSize=0.035) (data from The World Bank):


<video controls src="data/videos/pde.mp4" />


### Line charts

If you need to plot a numerical variable over time, you might find line plots to be handy. You can add more series to the plot (i.e., more lines) to compare across different numerical variables over time as grouped by a categorical variable. When we add more series, we usually differentiate them using a different color for each one. Here's an example:

<img src="data/images/line_chart.jpg" width="500"/>

### Tables and scorecards

The last visualizations we'll cover aren't actually plots. You can always represent your data in tabular form, and that's sometimes more appropriate than a visual chart. The general principle is that you should keep tables simple and uncluttered. If that's not possible, then try some of the other visualizations explained above.

A common use case of tables is when you need to do a **cross-tabulation (crosstab)** of two categorical variables. Crosstabs (a.k.a. contingency tables), count the number of elements of category A that also belong to category B and present the results in a table:

| Type | Physical | Online |
| --- | --- | --- |
| Guitars | 40 | 60 |
| Acc. Pianos | 30 | 50 |

This table shows that there were 40 guitars sold in a physical location. To learn more about contingency tables, check out [this](https://en.wikipedia.org/wiki/Contingency_table).

You can display a crosstab as-is in your dashboard if you like, but you can also slice it with a filter and then plot the relevant slice as a bar chart or a pie chart. Let's say you use a filter to slice out "online" (i.e., you only want to see sales from physical locations), which means the resulting plot can be:

<img src="data/images/physical_crosstab_plot.jpg" width="500"/>

In addition to these alternatives, you can also display single values using **scorecards**, like this:

<img src="data/images/scorecard.jpg" width="100"/>

## Applying our knowledge

### Exercise 2

Now that you know what kinds of visualizations go well with what kinds of data, it's time to choose which ones we'll use. Here's a table showing the variables that management requested to have in the dashboard. What are their data types?

| Variable | Type |
|-|-|
| Location |  |
| State of delivery |  |
| Time |  |
| Customer age group |  |
| Product type |  |
| Product description |  |
| Product line |  |

**Answer.**

The numerical variables that our report will calculate and display are:

| Variable | Type |
|-|-|
| Revenue | Numeric (float) |
| Profit | Numeric (float) |
| Items sold | Numeric (integer) |
| Number of customers | Numeric (integer) |

These are called the **metrics** of our dashboard. You can say that metrics are numbers that are calculated, and dimensions are filters that we use to determine which subsets of the data are used to compute those metrics:

<img src="data/images/metrics_and_dimensions.jpg" width="600"/>

The mathematical operations dashboards typically (but not exclusively) perform to create the metrics are:

1. **Sum**: sums all the values of the rows of the selected column
2. **Count**: counts how many rows there are in the selected column
3. **Count Distinct**: counts how many non-duplicate rows there are in the selected column
4. **Average**: finds the arithmetic mean of all the values of the rows of that column
5. **Minimum**: finds the minimum of all the values of the rows of that column
6. **Maximum**: finds the maximum of all the values of the rows of that column

### Exercise 3

What mathematical operations are most appropriate to:

1. Calculate the total number of items sold
2. Calculate the total number of customers who purchased something in a given time period
3. Calculate total profit
4. Calculate total revenue

**Answer.**

### Exercise 4

This is a table with all the possible pairwise combinations of dimensions. For each combination, it is possible to compute one or more metrics. For instance, with "Location" and "Customer age group", we can report how many items were sold by location and age group, how much profit each age bracket produces in each location, or even how many customers from each age group buy in that location.

Please replace the `?`s in this table with suitable visualization options (just filling the upper half is enough, as the bottom half is functionally equivalent). Use the [`visualization_exercise.xlsx`](/case.dashboard_design_fellow/files/data/visualization_exercise.xlsx) Excel file to record your answers.

| var/var | Location | State of delivery | Customer age group | Product type | Product description | Product line |
|-|-|-|-|-|-|-|
| Location | * | ? | ? | ? | ? | ? |
| State of delivery | &nbsp; | * | ? | ? | ? | ? |
| Customer age group | &nbsp; | &nbsp; | * | ? | ? | ? |
| Product type | &nbsp; | &nbsp; | &nbsp; | * | ? | ? |
| Product description | &nbsp; | &nbsp; | &nbsp; | &nbsp; | * | ? |
| Product line | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | * |

**Answer.**

### Exercise 5

Let's say we want to plot revenue, profit, and sales over time. What would be an appropriate visualization?

**Answer.**

## Planning out the layout of our dashboard

The table above was just a list of pairwise combinations of variables, but having a bunch of visualizations with no logical order whatsoever won't be pleasing to the eye and, most importantly, won't convey any useful information.

When the end user opens the dashboard, they should see a clear, self-descriptive roadmap of what it has to offer. A good dashboard is like a restaurant menu that categorizes the meals, arranges them in hierarchies according to what the customer wants to read first, and provides only the information needed to make a decision. If the customer wants to learn more about a dish (e.g., the specific ingredients used or the name of the chef), they would have to ask the waiter or waitress. Likewise, your dashboard should present the most relevant information first and show more detailed information only if the user asks for it.

Keep the following image in mind; it will help us as we move forward:

<img src="data/images/menu.jpg" width="500"/>

A good way to organize our dashboard is by creating **pages**. These are screens that show different aspects of your data, and the end user can switch between them by using a menu bar. Let's start with a basic set of pages - a "Bird's-Eye View" page (with an overview of the metrics) and a "Detailed View" page (which has more information and more filters).

Our dashboard will have three main layout components - a sidebar, a banner, and a body:

<img src="data/images/main_layout.jpg" width="500"/>

The sidebar will contain links to the pages. The banner will have the title of the page and optionally notes, date filters, and scorecards. The page body will contain the visualizations.

### The "Bird's-Eye View" page layout

Recall the original business problem. The managers want to know the following:

1. How have our sales, revenue, and profits evolved over time?
2. What are the locations with the highest numbers of sales and profit?
3. What are the items with the highest numbers of sales, profit, and revenue across product description, product type, product line, location, customer age group, and state of delivery?

The third question has lots of dimensions, so we can put it in the "Detailed View" page. Let's put the first two questions in the "Bird's-Eye View" page.

We know that the first question can be answered using line plots. Since revenue and profits are both measured in dollars, we will show both of them in a single-line plot using colored series like this:

![A line plot](data/images/first_question_line_plot.png "A line plot")

Sales are measured in number of items, so they should have a line plot of their own. But what about the second question?

### Exercise 6

What visualization would be appropriate to answer the second question?

> 2. What are the locations with the highest numbers of sales and profit?

**Answer.**

It would also be nice to give our users a summary of the whole page in a small set of aggregations. We can use scorecards for that. Let's create a scorecard for "Total revenue", one for "Total profit", and one for "Total sales", since these are the main metrics the managers are interested in.

### Exercise 7

The "Bird's-Eye View" page must contain the following elements:

* The company logo
* A date picker (to filter the data)
* The profit and revenue line chart
* The sales line chart
* The scorecards
* The profit and revenue bar chart
* The sales bar chart

Grab a piece of paper (or your computer) and draw the page layout you think would be most appropriate. Then take a picture or screenshot of your solution and post it so that the entire class can see it.

**Answer.**

### The "Detailed View" page layout

Let's now move on to the third question. Since we want to know sales and total profit for each item, it makes sense to have a chart for sales (measured in number of items) and a chart for profit (measured in dollars). The count of items and the sum of dollars are both numeric (the former is an integer, the latter is a float). As item names are categorical, bar charts would be fine. (Pie charts might not be easy to interpret because there are many products, and thus, the charts would get crowded).

If you recall the `products` table, you'll remember that each product has a line ("Musical instruments"), a type ("Guitars"), and a description ("Fender American Stratocaster"). To avoid having to create a page-level filter for each one of these variables, we can add them to a single "Items" chart as **drill-down** variables. Drill-down allows you to select one element of a chart and move to a lower level in the hierarchy. The following video may be helpful to understand this concept than a mere description in words (this is a bar chart of sales by item, with drill-down by line, type, and description):


<video controls src="data/videos/drill_down.mp4" />

We can have a bar chart like this and next to it another bar chart (with no drill-down this time) that shows total profit for each *item*. We want our drill-down in the first chart to filter the second chart so that when someone drills down by "Guitars", the second chart only shows guitars.

### Exercise 8

Now that we have the two main charts of the "Detailed View" page, let's add some secondary charts (which will be smaller in size to reflect their auxiliary status) for the dimensions that our management wants to filter the data by. These include location (number of items sold in each location), customer age group (number of customers in each age bracket), and state of delivery (number of items delivered). Which three plots would go well with these dimensions?

**Answer.**

### Exercise 9

Draw a layout that you think would be appropriate for the "Detailed View" page. Make sure to indicate what types of charts would be located where and what information those charts would show. Take a photo or a screenshot and share it with the class.

**Hint:** The "Detailed View" page can also include some scorecards and a date picker.

**Answer.**

## Conclusions

In this case, we looked at different types of visualizations and studied what kinds of data they are best suited to presenting. But we also saw that creating an analytical dashboard goes beyond this, and involves thinking about the concrete business questions that end users want to answer with the help of the dashboard. That is why we started by getting a grasp of the business process and the relevant dimensions and metrics, which informed the layout design and the choice of visualizations.

## Takeaways

Dashboard creation should always start with a sound understanding of the business process and the business questions. You also need to be familiar with the different types of visualizations and their strengths and weaknesses, and keep some basic design principles in mind as well. This allows you to build a dashboard that is functional but also not cluttered. Some key ideas you might want to keep in mind for the future are:

* *Dashboards are like restaurant menus.* They must be clear, straightforward and avoid overwhelming the user with too much information. Don't forget BLUF: Bottom Line Up Front. Put the most important details first and foremost, where it's easiest to read.
* *Your visualizations should be organized hierarchically.* The visuals that answer the most important questions come first. Visuals dedicated to answering secondary questions can be placed more to the right or bottom and be presented in a smaller size
* *Each visualization must answer a business question*, so it is always a good practice to label the chart or table with the question/purpose. You should also add explanatory text when you feel your users would appreciate it. However, if you find yourself adding too much text, then perhaps it is because your dashboard is not clear enough by itself, and you should re-evaluate your layout or visualizations

Additionally, drill-down and charts that double as page-level filters can save you a lot of space because you don't need to add more controls (like drop-down lists) to act as filters in the page. Not everything should be drillable though, because the dashboard should be kept as simple as possible.

## Attribution

"Cello study", Michael Maggs, April 13th, 2018, [Creative Commons Attribution Share Alike Unported 3.0](https://creativecommons.org/licenses/by-sa/3.0/deed.en), https://commons.wikimedia.org/wiki/File:Cello_study.jpg

"Platform Dashboard", Joël Jean Henry, 11 October 2019, Creative Commons Attribution-Share Alike 4.0 International license, https://commons.wikimedia.org/wiki/File:Platform_Dashboard.svg

"Marketing dashboard", HelicalInsight OpenSourceBI, 12 May 2015, Creative Commons Attribution-Share Alike 4.0 International license, https://commons.wikimedia.org/wiki/File:Marketing_dashboard.png

"Screenshot Dashboard", Wiko Bausoftware GmbH, 16 May 2019, Creative Commons Attribution-Share Alike 4.0 International license, https://commons.wikimedia.org/wiki/File:Screenshot_Dashboard.png

"American restaurant menu", Eric Fischer, April 14th, 2018, [Creative Commons Attribution 2.0 generic license](https://creativecommons.org/licenses/by/2.0/deed.en), https://commons.wikimedia.org/wiki/File:American_restaurant_menu_(30085586928).jpg#file


"Example of Scatter Plot", Dcbmariano, 19 April 2018, Creative Commons Attribution-Share Alike 4.0 International license, https://commons.wikimedia.org/wiki/File:Example_of_Scatter_Plot.jpg